In [1]:
import requests
import zipfile
import io
import pandas as pd
from datetime import datetime

In [2]:
# URL de base des archives Binance
BASE_URL = "https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1h/"

In [3]:
def download_month(year: int, month: int):
    """Télécharge les données de trading BTCUSDT pour un mois donné.
    
    Args:
        year (int): Année (ex: 2023)
        month (int): Mois (1-12)
    
    Returns:
        pd.DataFrame: DataFrame contenant les données de trading, ou None si le fichier n'existe pas.
    """
    fname = f"BTCUSDT-1h-{year}-{month:02d}.zip"
    url = BASE_URL + fname
    r = requests.get(url)
    if r.status_code == 200:
        z = zipfile.ZipFile(io.BytesIO(r.content))
        csv_file = z.namelist()[0]
        df = pd.read_csv(
            z.open(csv_file), 
            header=None,
            names=[
                "open_time", "open", "high", "low", "close", "volume",
                "close_time", "quote_asset_volume", "number_of_trades",
                "taker_buy_base", "taker_buy_quote", "ignore"
            ]
        )
        return df
    else:
        print(f"Pas trouvé : {url}")
        return None

In [4]:
def build_dataset(start_year=2017, start_month=8, end_year=None, end_month=None, out_csv="BTCUSDT_1h.csv"):
    """Télécharge et concatène toutes les données disponibles en un CSV unique."""
    if end_year is None or end_month is None:
        now = datetime.utcnow()
        end_year, end_month = now.year, now.month

    dfs = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if (year == start_year and month < start_month) or (year == end_year and month > end_month):
                continue
            print(f"Téléchargement {year}-{month:02d} ...")
            df = download_month(year, month)
            if df is not None:
                dfs.append(df)

    if dfs:
        all_data = pd.concat(dfs, ignore_index=True)
        # Conversion timestamp en datetime lisible
        all_data["open_time"] = pd.to_datetime(all_data["open_time"], unit="ms", errors="coerce")
        all_data["close_time"] = pd.to_datetime(all_data["close_time"], unit="ms", errors="coerce")
        
        # Delete corrupted rows
        # all_data.dropna(inplace=True, subset=["open_time", "close_time"])
        # Sauvegarde CSV
        all_data.to_csv(out_csv, index=False)
        print(f"\n✅ Dataset sauvegardé dans {out_csv} ({len(all_data)} lignes)")
        return all_data
    else:
        print("❌ Aucun fichier téléchargé.")
        return None

In [5]:
# Exemple d’utilisation : télécharger toutes les données depuis août 2017
df = build_dataset(start_year=2017, start_month=8, out_csv="../raw/BTCUSDT_1h_full.csv")

/tmp/ipykernel_4026/123367092.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Téléchargement 2017-08 ...
Téléchargement 2017-09 ...
Téléchargement 2017-10 ...
Téléchargement 2017-11 ...
Téléchargement 2017-12 ...
Téléchargement 2018-01 ...
Téléchargement 2018-02 ...
Téléchargement 2018-03 ...
Téléchargement 2018-04 ...
Téléchargement 2018-05 ...
Téléchargement 2018-06 ...
Téléchargement 2018-07 ...
Téléchargement 2018-08 ...
Téléchargement 2018-09 ...
Téléchargement 2018-10 ...
Téléchargement 2018-11 ...
Téléchargement 2018-12 ...
Téléchargement 2019-01 ...
Téléchargement 2019-02 ...
Téléchargement 2019-03 ...
Téléchargement 2019-04 ...
Téléchargement 2019-05 ...
Téléchargement 2019-06 ...
Téléchargement 2019-07 ...
Téléchargement 2019-08 ...
Téléchargement 2019-09 ...
Téléchargement 2019-10 ...
Téléchargement 2019-11 ...
Téléchargement 2019-12 ...
Téléchargement 2020-01 ...
Téléchargement 2020-02 ...
Téléchargement 2020-03 ...
Téléchargement 2020-04 ...
Téléchargement 2020-05 ...
Téléchargement 2020-06 ...
Téléchargement 2020-07 ...
Téléchargement 2020-08 ...
T

In [6]:
duplicates = df["open_time"].duplicated().sum()
print(f"Nombre de doublons dans open_time : {duplicates}")

df = df.drop_duplicates(subset=["open_time"], keep="first")

duplicates = df["open_time"].duplicated().sum()
print(f"Nombre de doublons dans open_time : {duplicates}")

df.info()

Nombre de doublons dans open_time : 5831
Nombre de doublons dans open_time : 0
<class 'pandas.core.frame.DataFrame'>
Index: 64526 entries, 0 to 64525
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   open_time           64525 non-null  datetime64[ns]
 1   open                64526 non-null  float64       
 2   high                64526 non-null  float64       
 3   low                 64526 non-null  float64       
 4   close               64526 non-null  float64       
 5   volume              64526 non-null  float64       
 6   close_time          64525 non-null  datetime64[ns]
 7   quote_asset_volume  64526 non-null  float64       
 8   number_of_trades    64526 non-null  int64         
 9   taker_buy_base      64526 non-null  float64       
 10  taker_buy_quote     64526 non-null  float64       
 11  ignore              64526 non-null  float64       
dtypes: datetime64[ns](2), float6

In [7]:
df = df.set_index("open_time").sort_index()

full_range = pd.date_range(df.index.min(), df.index.max(), freq="H")
df = df.reindex(full_range)

df.index.name = "open_time"

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64652 entries, 2017-08-17 04:00:00 to 2024-12-31 23:00:00
Freq: h
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   open                64482 non-null  float64       
 1   high                64482 non-null  float64       
 2   low                 64482 non-null  float64       
 3   close               64482 non-null  float64       
 4   volume              64482 non-null  float64       
 5   close_time          64482 non-null  datetime64[ns]
 6   quote_asset_volume  64482 non-null  float64       
 7   number_of_trades    64482 non-null  float64       
 8   taker_buy_base      64482 non-null  float64       
 9   taker_buy_quote     64482 non-null  float64       
 10  ignore              64482 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 5.9 MB


/tmp/ipykernel_4026/2554304782.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(df.index.min(), df.index.max(), freq="H")


## Add and transform for usefull features

In [8]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_features(df):
    df = df.copy()
    
    # SMA
    df["sma_7d"] = df["close"].rolling(window=7*24).mean()
    df["sma_30d"] = df["close"].rolling(window=30*24).mean()
    df["sma_50d"] = df["close"].rolling(window=50*24).mean()
    df["sma_100d"] = df["close"].rolling(window=100*24).mean()
    
    # Volatibility
    df["return"] = df["close"].pct_change()

    # Volatility on 20 hours
    df["volatility_20"] = df["return"].rolling(window=20).std()
    df["volatility_50"] = df["return"].rolling(window=50).std()
    df["volatility_100"] = df["return"].rolling(window=100).std()
    df["volatility_14d"] = df["return"].rolling(window=14*24).std()
    
    # RSI 14 and 14 days
    df["rsi_14"] = compute_rsi(df["close"], window=14)
    df["rsi_14d"] = compute_rsi(df["close"], window=14*24)
    
    # MACD
    df = compute_MACD(df)
    
    # Relative volume 20
    df["volume_sma20"] = df["volume"].rolling(window=20).mean()
    df["volume_sma20d"] = df["volume"].rolling(window=20*24).mean()
    df["volume_rel20"] = df["volume"] / df["volume_sma20"]
    df["volume_rel20d"] = df["volume"] / df["volume_sma20d"]
    
    return df

def compute_MACD(df):
    df = df.copy()
    
    # EMA 12 et EMA 26
    df["ema_12d"] = df["close"].ewm(span=12*24, adjust=False).mean()
    df["ema_26d"] = df["close"].ewm(span=26*24, adjust=False).mean()
    
    # MACD line
    df["MACD"] = df["ema_12d"] - df["ema_26d"]

    # Signal line (EMA 9 du MACD)
    df["Signal"] = df["MACD"].ewm(span=9*24, adjust=False).mean()

    # Histogramme
    df["MACD_Hist"] = df["MACD"] - df["Signal"]

    return df

In [9]:
df = compute_features(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64652 entries, 2017-08-17 04:00:00 to 2024-12-31 23:00:00
Freq: h
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   open                64482 non-null  float64       
 1   high                64482 non-null  float64       
 2   low                 64482 non-null  float64       
 3   close               64482 non-null  float64       
 4   volume              64482 non-null  float64       
 5   close_time          64482 non-null  datetime64[ns]
 6   quote_asset_volume  64482 non-null  float64       
 7   number_of_trades    64482 non-null  float64       
 8   taker_buy_base      64482 non-null  float64       
 9   taker_buy_quote     64482 non-null  float64       
 10  ignore              64482 non-null  float64       
 11  sma_7d              59926 non-null  float64       
 12  sma_30d             48125 non-null  float64       
 13  sma

/tmp/ipykernel_4026/768829287.py:23: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["return"] = df["close"].pct_change()


In [10]:
def clean_dataset(df):
    df = df.copy()
    
    # Colonnes inutiles
    drop_cols = [
        "close_time", "ignore", 
        "ema_12d", "ema_26d", 
        "volume_sma20", "volume_sma20d"
    ]
    
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])
    
    # Suppression des lignes avec NaN restants
    df = df.dropna()
    
    print(f"✅ Dataset nettoyé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
    return df


In [11]:
cleaned_df = clean_dataset(df)

✅ Dataset nettoyé : 27203 lignes, 25 colonnes


In [12]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27203 entries, 2017-12-15 22:00:00 to 2024-12-31 23:00:00
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                27203 non-null  float64
 1   high                27203 non-null  float64
 2   low                 27203 non-null  float64
 3   close               27203 non-null  float64
 4   volume              27203 non-null  float64
 5   quote_asset_volume  27203 non-null  float64
 6   number_of_trades    27203 non-null  float64
 7   taker_buy_base      27203 non-null  float64
 8   taker_buy_quote     27203 non-null  float64
 9   sma_7d              27203 non-null  float64
 10  sma_30d             27203 non-null  float64
 11  sma_50d             27203 non-null  float64
 12  sma_100d            27203 non-null  float64
 13  return              27203 non-null  float64
 14  volatility_20       27203 non-null  float64
 15  volatility_50     

In [13]:
cleaned_df.to_parquet("../cleaned/BTC_features_clean.parquet")

--- 

## Treatmennt of the times series gaps

In [14]:
# Fixing the latences between hours in the close and open prices
def fix_gaps(df:pd.DataFrame) -> list[pd.DataFrame]:
    """Split a time-indexed DataFrame into continuous segments where hourly gaps are detected.

    Args:
        df (pd.DataFrame): A DataFrame with a sorted DatetimeIndex (assumed hourly frequency),
            typically containing OHLC data, in which we want to detect and isolate time gaps
            longer than one hour.

    Returns:
        list[pd.DataFrame]: A list of DataFrames, each representing a continuous time series
            segment with no gaps exceeding one hour. Suitable for independent processing or
            analysis without temporal discontinuities.
    """
    # Supposons que sub_df est votre DataFrame avec un DatetimeIndex trié
    df = df.sort_index()

    # Étape 1 : détecter les gaps > 1h
    delta = df.index.to_series().diff()
    is_gap = delta > pd.Timedelta('1h')

    # Étape 2 : créer un groupe cumulatif avec cumsum()
    group_id = is_gap.cumsum()

    # Étape 3 : découper en sous-DataFrames
    segments = [g for _, g in df.groupby(group_id)]

    print(f"Nombre de segments identifiés : {len(segments)}")
    
    return segments

In [15]:
# Separate whole data into different continuous segments
continue_dfs = fix_gaps(cleaned_df)
print(continue_dfs[0].tail())
print(continue_dfs[1].head())

Nombre de segments identifiés : 8
                         open      high       low     close       volume  \
open_time                                                                  
2018-01-03 23:00:00  14792.32  14940.82  14710.00  14919.51   648.924200   
2018-01-04 00:00:00  14919.51  14948.00  14710.25  14753.76   820.846740   
2018-01-04 01:00:00  14765.65  15179.00  14649.99  15172.12  1181.627263   
2018-01-04 02:00:00  15175.00  15280.00  15059.00  15110.90  1055.426462   
2018-01-04 03:00:00  15110.90  15119.98  15110.90  15119.97     4.635181   

                     quote_asset_volume  number_of_trades  taker_buy_base  \
open_time                                                                   
2018-01-03 23:00:00        9.593982e+06            6624.0      527.040262   
2018-01-04 00:00:00        1.214946e+07            6940.0      608.858344   
2018-01-04 01:00:00        1.752356e+07            9267.0      844.362806   
2018-01-04 02:00:00        1.605312e+07         

In [16]:
def reassign_close_to_open(df: pd.DataFrame) -> pd.DataFrame:
    """Reassign the 'open' price of each hour to be the 'close' price of the previous hour.

    Args:
        df (pd.DataFrame): DataFrame with OHLC data and a DatetimeIndex.

    Returns:
        pd.DataFrame: DataFrame with updated 'open' prices.
    """
    df = df.copy()
    df["shifted_close"] = df["close"].shift(1)
    df["open"] = df["shifted_close"]
    df.drop(columns=["shifted_close"], inplace=True)
    df.dropna(inplace=True)
    return df

In [17]:
# reassing close to open for each segment
for i, df_elemen in enumerate(continue_dfs):
    continue_dfs[i] = reassign_close_to_open(df_elemen)

---

## Treatment of the target

In [18]:
def define_future_evolution(df: pd.DataFrame) -> pd.DataFrame:
    """Define a boolean target indicating if the price will go up in the next hour.

    Args:
        df (pd.DataFrame): DataFrame with OHLC data.

    Returns:
        pd.DataFrame: DataFrame with an additional 'will_up' column.
    """
    df = df.copy()
    df["will_up"] = (df["close"].shift(-1) > df["close"]).astype(bool)
    return df

In [19]:
for i, df_element in enumerate(continue_dfs):
    continue_dfs[i] = define_future_evolution(df_element)

In [20]:
continue_dfs[0][["open", "close", "will_up"]].head(20)

,open,close,will_up
open_time,,,
2017-12-15 23:00:00,17502.72,17539.83,True
2017-12-16 00:00:00,17539.83,17577.99,False
2017-12-16 01:00:00,17577.99,17460.00,False
2017-12-16 02:00:00,17460.00,17350.00,False
2017-12-16 03:00:00,17350.00,17250.00,True
2017-12-16 04:00:00,17250.00,17437.06,True
2017-12-16 05:00:00,17437.06,17489.00,True
2017-12-16 06:00:00,17489.00,17625.88,True
2017-12-16 07:00:00,17625.88,17677.54,True


---
## Treating last segment to get validation and test data

In [34]:
continue_dfs_copy = continue_dfs.copy()

last_df = continue_dfs_copy[-1]
total_len = sum(len(d) for d in continue_dfs_copy)
print(f"Total rows across all segments: {total_len}")
print(f"Row in last df: {len(last_df)}")

ideal_size_valtest = int(total_len * 0.15)
print(f"ideal size of val and test set (15% each): {ideal_size_valtest}")
test_df = last_df.iloc[-ideal_size_valtest:]
val_df = last_df.iloc[-2*ideal_size_valtest:-ideal_size_valtest] 
last_seg_reshaped = last_df.iloc[:-2*ideal_size_valtest]

print("")
print(f"Size of val set: {len(val_df)}")
print(f"Size of test set: {len(test_df)}")
print(f"Size of reshaped last segment: {len(last_seg_reshaped)}")
print(f"Size of all those segments: {len(val_df) + len(test_df) + len(last_seg_reshaped)}")


# Replace last segment with reshaped one and add val and test sets
continue_dfs_copy[-1] = last_seg_reshaped
continue_dfs_copy.append(val_df)
continue_dfs_copy.append(test_df)

print("")
print("Total size of all the segments after replacement:" , sum(len(d) for d in continue_dfs_copy))

Total rows across all segments: 27195
Row in last df: 13162
ideal size of val and test set (15% each): 4079

Size of val set: 4079
Size of test set: 4079
Size of reshaped last segment: 5004
Size of all those segments: 13162

Total size of all the segments after replacement: 27195


In [35]:
# Validation of the changes

continue_dfs = continue_dfs_copy

---
## Saving all the dataframes

In [39]:
def save_segments(segments: list[pd.DataFrame], base_filename="BTCUSDT_1h_continuous") -> None:
    """Save each DataFrame segment to a separate CSV file. Last files will be named val and test respectively.

    Args:
        segments (list[pd.DataFrame]): List of DataFrame segments to save.
        base_filename (str): Base filename for the output CSV files.
    """
    for i, segment in enumerate(segments):
        filename = f"{base_filename}_{i+1}.csv"
        if i == len(segments) - 2:
            filename = f"{base_filename}_val.csv"
        elif i == len(segments) - 1:
            filename = f"{base_filename}_test.csv"
        segment.to_csv(filename)
        print(f"Segment {i+1} sauvegardé dans {filename}")

In [40]:
save_segments(continue_dfs, base_filename="../prepared/BTCUSDT_1h_continuous")

Segment 1 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_1.csv
Segment 2 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_2.csv
Segment 3 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_3.csv
Segment 4 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_4.csv
Segment 5 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_5.csv
Segment 6 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_6.csv
Segment 7 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_7.csv
Segment 8 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_8.csv
Segment 9 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_val.csv
Segment 10 sauvegardé dans ../prepared/BTCUSDT_1h_continuous_test.csv
